Задание 4. Итеративные методы решения СЛАУ

In [20]:
!pip3 install numpy
!pip3 install scipy

In [21]:
import numpy as np
from scipy.linalg import hilbert

Реализация методов решения СЛАУ

In [22]:
def simple_iterations(a, b, eps = 1e-5):
    MAX_ITERATIONS = 50000
    _a = np.zeros((len(a), len(a)))
    _b = np.array([ 0. ] * len(b))
    for i in range(len(a)):
        for j in range(len(a)):
            if i == j:
                _a[i][j] = 0.
            else:
                assert not np.isclose(a[i][i], 0), "Диагональный элемент a равен 0!"
                _a[i][j] = -a[i][j] / a[i][i]
    for i in range(len(b)):
        assert not np.isclose(a[i][i], 0), "Диагональный элемент a равен 0!"
        _b[i] = b[i] / a[i][i]
    x_prev = np.copy(_b)
    x_cur = np.copy(_b)
    iterations = 1
    x_cur = np.add(np.dot(_a, x_cur), _b)
    while np.linalg.norm(x_cur - x_prev) > eps:
        x_prev = np.copy(x_cur)
        x_cur = np.add(np.dot(_a, x_cur), _b)
        iterations += 1
        if iterations == MAX_ITERATIONS:
            print("МЕТОД НЕ СХОДИТСЯ")
            return x_cur, iterations
    return x_cur, iterations

def seidel(a, b, eps = 1e-5):
    MAX_ITERATIONS = 50000
    x_cur = np.zeros(len(b))
    iterations = 0
    while True:
        x_prev = np.copy(x_cur)
        for i in range(len(a)):
            s1 = sum(a[i][j] * x_cur[j] for j in range(i))
            s2 = sum(a[i][j] * x_prev[j] for j in range(i + 1, len(b)))
            assert not np.isclose(a[i][i], 0), "Диагональный элемент равен 0!"
            x_cur[i] = (b[i] - s1 - s2) / a[i][i]
        iterations += 1
        if iterations == MAX_ITERATIONS:
            print("\tМЕТОД НЕ СХОДИТСЯ")
            return x_cur, iterations
        if np.linalg.norm(x_cur - x_prev) < eps:
            break
    return x_cur, iterations

Тестирование метода простой итерации

In [23]:
def test_simple_iterations(a, b, eps):
    print("\tТочность:", eps)
    res = simple_iterations(a, b, eps)
    print("\tМетод простой итерации: {} (количество итераций: {})".format(res[0], res[1]))
    print("\tРешение верно:", np.allclose(np.dot(a, res[0]), b))

print("Тестирование на матрице Гильберта")
N = 5
a = hilbert(N)
b = np.dot(a, np.array([ 1 ] * N))
test_simple_iterations(a, b, 1e-7)

print("Тестирование на матрице из методички А. Н. Пакулиной")
a = np.array([
    [-401.64, 200.12],
    [1200.72, -601.76]
])
b = np.dot(a, np.array([ 1 ] * len(a)))
test_simple_iterations(a, b, 1e-3)
test_simple_iterations(a, b, 1e-6)
test_simple_iterations(a, b, 1e-9)

Тестирование на матрице Гильберта
	Точность: 1e-07
	Метод простой итерации: [inf nan nan nan nan] (количество итераций: 574)
	Решение верно: False
Тестирование на матрице из методички А. Н. Пакулиной
	Точность: 0.001
	Метод простой итерации: [0.99955471 0.99821677] (количество итераций: 2412)
	Решение верно: False
	Точность: 1e-06
	Метод простой итерации: [0.99999956 0.99999822] (количество итераций: 4786)
	Решение верно: True
	Точность: 1e-09
	Метод простой итерации: [1. 1.] (количество итераций: 7158)
	Решение верно: True


Тестирование метода Зейделя

In [28]:
def test_seidel(a, b, eps):
    print("\tТочность:", eps)
    res = seidel(a, b, eps)
    print("\tМетод простой итерации: {} (количество итераций: {})".format(res[0], res[1]))
    print("\tРешение верно:", np.allclose(np.dot(a, res[0]), b))

print("Тестирование на матрице Гильберта")
N = 5
a = hilbert(N)
b = np.dot(a, np.array([ 1 ] * N))
test_seidel(a, b, 1e-6)

print("Тестирование на матрице из методички А. Н. Пакулиной")
a = np.array([
    [-401.64, 200.12],
    [1200.72, -601.76]
])
b = np.dot(a, np.array([ 1 ] * len(a)))
test_seidel(a, b, 1e-3)
test_seidel(a, b, 1e-6)
test_seidel(a, b, 1e-9)

Тестирование на матрице Гильберта
	Точность: 1e-06
	Метод простой итерации: [0.99984558 1.00283474 0.987923   1.01808116 0.99121212] (количество итераций: 14229)
	Решение верно: True
Тестирование на матрице из методички А. Н. Пакулиной
	Точность: 0.001
	Метод простой итерации: [0.92354145 0.84743867] (количество итераций: 323)
	Решение верно: False
	Точность: 1e-06
	Метод простой итерации: [0.99992368 0.99984771] (количество итераций: 1510)
	Решение верно: True
	Точность: 1e-09
	Метод простой итерации: [0.99999992 0.99999985] (количество итераций: 2696)
	Решение верно: True


Сравнение количества итераций

In [25]:
print("Сравнение количества итераций")
res1 = simple_iterations(a, b, 1e-9)
print("\tМетод простой итерации:", res1[1])
res2 = seidel(a, b, 1e-9)
print("\tМетод Зейделя:", res2[1])

Сравнение количества итераций
	Метод простой итерации: 7158
	Метод Зейделя: 2696
